In [37]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_userseg', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43319)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", 

Spark master launched!
Creating new Spark session, name: west0_userseg...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:40


In [36]:
sphynx.stop()

In [7]:
# 각 2month 
pre_start_date = "2022-05-11"
pre_end_date = "2022-07-12"

post_start_date = "2022-07-13"
post_end_date = "2022-09-06"

## PRE

In [143]:
pre_gcoin = load_data_mart("pc", pre_start_date, pre_end_date, "gcoin_use")

In [144]:
pre_wsus_gcoin = pre_gcoin.where(col("event_name").like("%wsus%"))
pre_wsus_user = pre_wsus_gcoin.select("account_id").distinct()

In [145]:
pre_costume = pre_gcoin.where((~col("event_name").like("%wsus%")) & (~col("event_name").like("%survivorpass%")) & (~col("event_name").like("%yourshop%"))) \
    .where((col("sub_category").isin(["costume", "gear", "vehicle"])) | ((col("sub_category") == "crate") ))
pre_costume_user = pre_costume.select("account_id").distinct()

In [54]:
pre_costume.groupBy("event_type", "sub_category").agg(count("*")).show(truncate=False)

+-------------+------------+--------+
|event_type   |sub_category|count(1)|
+-------------+------------+--------+
|other        |costume     |332827  |
|other        |crate       |25669   |
|streamer     |crate       |177325  |
|streamer     |costume     |97236   |
|collaboration|crate       |249867  |
|collaboration|costume     |686     |
|other        |vehicle     |2799    |
|other        |costume     |68179   |
|other        |gear        |134724  |
|esports      |crate       |32154   |
|other        |crate       |214454  |
+-------------+------------+--------+



In [56]:
pre_costume.where(col("sub_category")=="crate").groupBy("event_type", "sub_category", "product_name").agg(count("*")).show(truncate=False)

+-------------+------------+-----------------------------+--------+
|event_type   |sub_category|product_name                 |count(1)|
+-------------+------------+-----------------------------+--------+
|streamer     |crate       |RASH BEIXI'S LION COSTUME SET|19417   |
|other        |crate       |BOOKSTACK BACKPACK PACK      |2721    |
|collaboration|crate       |COMMANDO SWEATSHIRT          |854     |
|other        |crate       |SPRING EVENING OUTFIT SET    |5975    |
|collaboration|crate       |FRESH HOODS CREW CROPPED PACK|4094    |
|other        |crate       |SPRING WALK OUTFIT SET       |6469    |
|collaboration|crate       |CAUTION HOODIE               |2685    |
|other        |crate       |DESPERADO COSTUME SET        |6642    |
|collaboration|crate       |SKYDIVER HOODIE              |1120    |
|collaboration|crate       |STEALTH SWEATPANTS           |2331    |
|collaboration|crate       |GORILLA FRESH NECK SLEEVE    |2301    |
|other        |crate       |BUNNY EXPRESS UNIFOR

In [146]:
pre_costume = pre_costume.withColumn("patch", when(col("date") < "2022-06-08", lit("17.2")).otherwise(lit("18.1")))
pre_wsus_gcoin = pre_wsus_gcoin.withColumn("patch", when(col("date") < "2022-06-08", lit("17.2")).otherwise(lit("18.1")))

In [147]:
pre_only_costume_gcoin = pre_costume.join(pre_wsus_user, "account_id", "leftanti")
pre_costume_and_wsus_gcoin = pre_wsus_gcoin.join(pre_costume_user, "account_id")
pre_only_wsus_gcoin= pre_wsus_gcoin.join(pre_costume_user, "account_id", "leftanti")

In [148]:
from pyspark.sql.types import * 

def wsus_cnt(price):
    if price == 1800:
        return 10
    elif price == 200:
        return 1
    else:
        return 0
    
assign_wsus_cnt = udf(wsus_cnt, IntegerType())

In [149]:
pre_only_costume_gcoin = pre_only_costume_gcoin.withColumn("group", lit("a"))
pre_costume_and_wsus_gcoin = pre_costume_and_wsus_gcoin.withColumn("group", lit("b")).withColumn("wsus_cnt", assign_wsus_cnt("price"))
pre_only_wsus_gcoin = pre_only_wsus_gcoin.withColumn("group", lit("c")).withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [150]:
pre_only_costume_gcoin_by_user = pre_only_costume_gcoin.groupBy("group", "account_id") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_total_use"), (sum("paid_use")/2).alias("avg_paid_use"))

In [151]:
pre_costume_and_wsus_gcoin_by_user = pre_costume_and_wsus_gcoin.groupBy("group", "account_id") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_total_use"), (sum("paid_use")/2).alias("avg_paid_use"), (sum("wsus_cnt")/2).alias("avg_wsus_cnt"))

In [152]:
pre_only_wsus_gcoin_by_user = pre_only_wsus_gcoin.groupBy("group", "account_id") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_total_use"), (sum("paid_use")/2).alias("avg_paid_use"), (sum("wsus_cnt")/2).alias("avg_wsus_cnt"))

In [15]:
pre_only_costume_gcoin_by_user.show(truncate=False)

+-----+----------------------------------------+-------------+------------+
|group|account_id                              |avg_total_use|avg_paid_use|
+-----+----------------------------------------+-------------+------------+
|a    |account.001a4cdf858c4694bf2c8b0a2ea051a7|1390.0       |0.0         |
|a    |account.001a84b7c1f44ad6896aacd5f0e68589|300.0        |0.0         |
|a    |account.003b956ea1a842188a565d944c174454|300.0        |0.0         |
|a    |account.00593012c1c44c319d59b13a0e870ee0|1000.0       |0.0         |
|a    |account.008ec13cece342a190daa06556d0493a|990.0        |0.0         |
|a    |account.008ec284a54e4117a4ae663c80b68532|200.0        |0.0         |
|a    |account.008fa524a65a4d0fb89c518d54a55de4|1000.0       |50.0        |
|a    |account.00bb345b36ec405a99bf6c938cf16874|200.0        |0.0         |
|a    |account.00c090cc3e7f4a279e255f55da161255|990.0        |0.0         |
|a    |account.00d1973f6a6d4ccaa8614db57402d9b2|500.0        |0.0         |
|a    |accou

In [60]:
# pre_only_costume_gcoin_by_user = pre_only_costume_gcoin.groupBy("account_id", "group").agg(sum("free_use").alias("pre_free_use"), sum("paid_use").alias("pre_paid_use"))
# pre_costume_and_wsus_gcoin_by_user = pre_costume_and_wsus_gcoin.groupBy("account_id", "group").agg(sum("free_use").alias("pre_free_use"), sum("paid_use").alias("pre_paid_use"))
# pre_only_wsus_gcoin_by_user = pre_only_wsus_gcoin.groupBy("account_id", "group").agg(sum("free_use").alias("pre_free_use"), sum("paid_use").alias("pre_paid_use"))

### a (ONLY COSTUME)

In [153]:
pre_only_costume_gcoin_by_user = pre_only_costume_gcoin_by_user.withColumn("seg", \
                                         when(col("avg_total_use") <= 1000, "A") \
                                        .when((col("avg_total_use") > 1000) & (col("avg_total_use") <= 5000), "B") \
                                        .when((col("avg_total_use") > 5000) & (col("avg_total_use") <= 10000), "C") \
                                        .when((col("avg_total_use") > 10000) & (col("avg_total_use") <= 30000), "D") \
                                        .when((col("avg_total_use") > 30000), "E") \
                                     )

In [154]:
pre_only_costume_gcoin_by_user = pre_only_costume_gcoin_by_user.withColumn("avg_wsus_cnt", lit(0))

### b (COSTUME + WSUS)

In [155]:
pre_costume_and_wsus_gcoin_by_user = pre_costume_and_wsus_gcoin_by_user.withColumn("seg", \
                                    when(col("avg_wsus_cnt") <=3, "A") \
                                    .when((col("avg_wsus_cnt") > 3) & (col("avg_wsus_cnt") <= 15), "B") \
                                    .when((col("avg_wsus_cnt") > 15) & (col("avg_wsus_cnt") <= 30), "C") \
                                    .when((col("avg_wsus_cnt") > 30) & (col("avg_wsus_cnt") <= 240), "D") \
                                    .when((col("avg_wsus_cnt") > 240), "E") \
                                              )

### c (ONLY WSUS)

In [156]:
pre_only_wsus_gcoin_by_user = pre_only_wsus_gcoin_by_user.withColumn("seg", \
                                                when(col("avg_wsus_cnt") <=10, "A") \
                                                .when((col("avg_wsus_cnt") > 10) & (col("avg_wsus_cnt") <= 50), "B") \
                                                .when((col("avg_wsus_cnt") > 50) & (col("avg_wsus_cnt") <= 100), "C") \
                                                .when((col("avg_wsus_cnt") > 100) & (col("avg_wsus_cnt") <= 800), "D") \
                                                .when((col("avg_wsus_cnt") > 800), "E") \
                                                          )

### total pre

In [157]:
pre_df = pre_only_costume_gcoin_by_user.unionByName(pre_costume_and_wsus_gcoin_by_user).unionByName(pre_only_wsus_gcoin_by_user)

In [158]:
pre_df = pre_df.withColumnRenamed("group", "pre_group").withColumnRenamed("seg", "pre_seg")

In [159]:
pre_df.groupBy("pre_group", "pre_seg").agg(count("*")).show(truncate=False)

+---------+-------+--------+
|pre_group|pre_seg|count(1)|
+---------+-------+--------+
|c        |E      |12      |
|b        |B      |49071   |
|a        |B      |53120   |
|c        |B      |36897   |
|b        |A      |152433  |
|b        |E      |3963    |
|a        |C      |519     |
|c        |D      |4032    |
|b        |D      |30158   |
|c        |A      |880880  |
|a        |D      |11      |
|a        |A      |540246  |
|b        |C      |16045   |
|c        |C      |6879    |
+---------+-------+--------+



In [37]:
pre_df.printSchema()

root
 |-- pre_group: string (nullable = false)
 |-- account_id: string (nullable = true)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- pre_seg: string (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)



In [110]:
pre_df.count()

1774266

In [160]:
mysql.drop_table("labs", "workshop_user_pre_seg_tmp")
mysql.insert_table(pre_df, "labs", "workshop_user_pre_seg_tmp")

## POST

In [9]:
post_gcoin = load_data_mart("pc", post_start_date, post_end_date, "gcoin_use")

In [162]:
post_wsus_gcoin = post_gcoin.where(col("event_name").like("%wsus%"))
post_wsus_user = post_wsus_gcoin.select("account_id").distinct()

In [163]:
post_costume = post_gcoin.where((~col("event_name").like("%wsus%")) & (~col("event_name").like("%survivorpass%")) & (~col("event_name").like("%yourshop%"))) \
    .where((col("sub_category").isin(["costume", "gear", "vehicle"])) | ((col("sub_category") == "crate") ) | col("event_name").like("%workshop%"))
post_costume_user = post_costume.select("account_id").distinct()

In [164]:
post_costume = post_costume.withColumn("patch", when(col("date") < "2022-08-10", lit("18.2")).otherwise(lit("19.1")))
post_wsus_gcoin = post_wsus_gcoin.withColumn("patch", when(col("date") < "2022-08-10", lit("18.2")).otherwise(lit("19.1")))

In [165]:
post_only_costume_gcoin = post_costume.join(post_wsus_user, "account_id", "leftanti")
post_costume_and_wsus_gcoin = post_wsus_gcoin.join(post_costume_user, "account_id")
post_only_wsus_gcoin= post_wsus_gcoin.join(post_costume_user, "account_id", "leftanti")

In [166]:
def chest_cnt(product_id):
    if product_id in ["itemdesc.14300001", "itemdesc.14300002", "itemdesc.13000577", "itemdesc.13000580"]:
        return 1
    elif product_id in ["itemdesc.13000578", "itemdesc.13000581"]:
        return 5
    elif product_id == "itemdesc.13000582":
        return 10
    else:
        return 0

assign_chest_cnt = udf(chest_cnt, IntegerType())

### a (ONLY COSTUME)

In [167]:
post_only_costume_gcoin = post_only_costume_gcoin.withColumn("group", lit("a")) \
    .withColumn("costume_type", when(col("event_name").like("%workshop%"), lit("workshop")).otherwise(lit("store"))) \
    .withColumn("chest_cnt", when(col("event_name").like("%workshop%"), assign_chest_cnt(col("product_id"))).otherwise(lit(0))) \
    .withColumn("wsus_cnt", lit(0))

In [168]:
post_only_costume_gcoin_by_user = post_only_costume_gcoin.groupBy("group", "account_id", "costume_type") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_total_use"), (sum("paid_use")/2).alias("avg_paid_use"), (sum("chest_cnt")/2).alias("avg_chest_cnt"), (sum("wsus_cnt")/2).alias("avg_wsus_cnt")) 

In [169]:
post_only_costume_gcoin_by_user_for_seg = post_only_costume_gcoin.groupBy("account_id") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_all_costume_total_use")) \
    .withColumn("seg", \
     when(col("avg_all_costume_total_use") <= 1000, "A") \
    .when((col("avg_all_costume_total_use") > 1000) & (col("avg_all_costume_total_use") <= 5000), "B") \
    .when((col("avg_all_costume_total_use") > 5000) & (col("avg_all_costume_total_use") <= 10000), "C") \
    .when((col("avg_all_costume_total_use") > 10000) & (col("avg_all_costume_total_use") <= 30000), "D") \
    .when((col("avg_all_costume_total_use") > 30000), "E") \
 )
post_only_costume_gcoin_by_user = post_only_costume_gcoin_by_user.join(post_only_costume_gcoin_by_user_for_seg, "account_id")

In [58]:
post_only_costume_gcoin_by_user.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- group: string (nullable = false)
 |-- costume_type: string (nullable = false)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- avg_chest_cnt: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)
 |-- avg_all_costume_total_use: double (nullable = true)
 |-- seg: string (nullable = true)



### b (WSUS + COSTUME)

In [198]:
post_costume_and_wsus_gcoin_user = post_costume_and_wsus_gcoin.select("account_id").distinct()

post_costume_and_wsus_gcoin_a = post_costume.join(post_costume_and_wsus_gcoin_user, "account_id") \
    .withColumn("costume_type", when(col("event_name").like("%workshop%"), lit("workshop")).otherwise(lit("store"))) \
    .withColumn("chest_cnt", when(col("event_name").like("%workshop%"), assign_chest_cnt(col("product_id"))).otherwise(lit(0)))

# 우선 workshop만
post_costume_and_wsus_gcoin_a_by_user = post_costume_and_wsus_gcoin_a.where(col("costume_type") == "workshop").groupBy("account_id", "costume_type") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_total_use"), (sum("paid_use")/2).alias("avg_paid_use"), (sum("chest_cnt")/2).alias("avg_chest_cnt")) 

In [51]:
post_costume_and_wsus_gcoin_a_by_user.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- costume_type: string (nullable = false)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- avg_chest_cnt: double (nullable = true)



In [171]:
post_costume_and_wsus_gcoin_b = post_costume_and_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt(col("price")))

post_costume_and_wsus_gcoin_b_by_user = post_costume_and_wsus_gcoin_b.groupBy("account_id") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_wsus_total_use"), (sum("paid_use")/2).alias("avg_wsus_paid_use"), (sum("wsus_cnt")/2).alias("avg_wsus_cnt"))

In [199]:
post_costume_and_wsus_gcoin_a_by_user.select(count("*"), countDistinct("account_id")).show(truncate=False)

+--------+--------------------------+
|count(1)|count(DISTINCT account_id)|
+--------+--------------------------+
|245305  |245305                    |
+--------+--------------------------+



In [190]:
post_costume_and_wsus_gcoin_b_by_user.select(sum("avg_wsus_total_use")).show(truncate=False)

+-----------------------+
|sum(avg_wsus_total_use)|
+-----------------------+
|1.1351873E9            |
+-----------------------+



In [194]:
post_costume_and_wsus_gcoin_b_by_user.select(count("*"), countDistinct("account_id")).show(truncate=False)

+--------+--------------------------+
|count(1)|count(DISTINCT account_id)|
+--------+--------------------------+
|309914  |309914                    |
+--------+--------------------------+



In [60]:
post_costume_and_wsus_gcoin_b_by_user.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- avg_wsus_total_use: double (nullable = true)
 |-- avg_wsus_paid_use: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)



In [200]:
# post_costume_and_wsus_gcoin_a_by_user: workshop 사용
# post_costume_and_wsus_gcoin_b_by_user: wsus 사용
post_costume_and_wsus_gcoin_by_user = post_costume_and_wsus_gcoin_a_by_user.join(post_costume_and_wsus_gcoin_b_by_user, "account_id", "right")      

In [201]:
post_costume_and_wsus_gcoin_by_user.select(count("*"), countDistinct("account_id")).show(truncate=False)

+--------+--------------------------+
|count(1)|count(DISTINCT account_id)|
+--------+--------------------------+
|309914  |309914                    |
+--------+--------------------------+



In [172]:
# post_costume_and_wsus_gcoin_by_user = post_costume_and_wsus_gcoin_a_by_user.join(post_costume_and_wsus_gcoin_b_by_user, "account_id")

In [202]:
post_costume_and_wsus_gcoin_a_tmp = post_costume_and_wsus_gcoin_a.groupBy("account_id") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_all_costume_total_use"))

In [203]:
post_costume_and_wsus_gcoin_by_user = post_costume_and_wsus_gcoin_by_user.withColumn("seg", \
    when(col("avg_wsus_cnt") <= 3, "A") \
    .when((col("avg_wsus_cnt") > 3) & (col("avg_wsus_cnt") <= 15), "B") \
    .when((col("avg_wsus_cnt") > 15) & (col("avg_wsus_cnt") <= 30), "C") \
    .when((col("avg_wsus_cnt") > 30) & (col("avg_wsus_cnt") <= 240), "D") \
    .when((col("avg_wsus_cnt") > 240), "E") \
)

In [204]:
post_costume_and_wsus_gcoin_by_user = post_costume_and_wsus_gcoin_by_user.join(post_costume_and_wsus_gcoin_a_tmp, "account_id") \
    .withColumn("group", lit("b"))

In [205]:
post_costume_and_wsus_gcoin_by_user.select(count("*"), countDistinct("account_id")).show(truncate=False)

+--------+--------------------------+
|count(1)|count(DISTINCT account_id)|
+--------+--------------------------+
|309914  |309914                    |
+--------+--------------------------+



In [206]:
post_costume_and_wsus_gcoin_by_user.select(sum("avg_wsus_total_use")).show(truncate=False)

+-----------------------+
|sum(avg_wsus_total_use)|
+-----------------------+
|1.1351873E9            |
+-----------------------+



In [68]:
post_costume_and_wsus_gcoin_by_user.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- costume_type: string (nullable = false)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- avg_chest_cnt: double (nullable = true)
 |-- avg_wsus_total_use: double (nullable = true)
 |-- avg_wsus_paid_use: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)
 |-- seg: string (nullable = true)
 |-- avg_all_costume_total_use: double (nullable = true)
 |-- group: string (nullable = false)



### c (ONLY WSUS)

In [176]:
post_only_wsus_gcoin_by_user = post_only_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt(col("price"))) \
    .groupBy("account_id") \
    .agg((sum(col("free_use") + col("paid_use"))/2).alias("avg_wsus_total_use"), (sum("paid_use")/2).alias("avg_wsus_paid_use"), (sum("wsus_cnt")/2).alias("avg_wsus_cnt"))

In [177]:
post_only_wsus_gcoin_by_user = post_only_wsus_gcoin_by_user.withColumn("seg", \
            when(col("avg_wsus_cnt") <=10, "A") \
            .when((col("avg_wsus_cnt") > 10) & (col("avg_wsus_cnt") <= 50), "B") \
            .when((col("avg_wsus_cnt") > 50) & (col("avg_wsus_cnt") <= 100), "C") \
            .when((col("avg_wsus_cnt") > 100) & (col("avg_wsus_cnt") <= 800), "D") \
            .when((col("avg_wsus_cnt") > 800), "E") \
).withColumn("group", lit("c"))

In [178]:
post_only_wsus_gcoin_by_user = post_only_wsus_gcoin_by_user.withColumn("group", lit("c"))

In [72]:
post_only_wsus_gcoin_by_user.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- avg_wsus_total_use: double (nullable = true)
 |-- avg_wsus_paid_use: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)
 |-- seg: string (nullable = true)



In [191]:
post_only_wsus_gcoin_by_user.select(sum("avg_wsus_total_use")).show(truncate=False)

+-----------------------+
|sum(avg_wsus_total_use)|
+-----------------------+
|2.851415E8             |
+-----------------------+



### total

In [74]:
post_only_costume_gcoin_by_user.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- group: string (nullable = false)
 |-- costume_type: string (nullable = false)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- avg_chest_cnt: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)
 |-- avg_all_costume_total_use: double (nullable = true)
 |-- seg: string (nullable = true)



In [75]:
post_costume_and_wsus_gcoin_by_user.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- costume_type: string (nullable = false)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- avg_chest_cnt: double (nullable = true)
 |-- avg_wsus_total_use: double (nullable = true)
 |-- avg_wsus_paid_use: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)
 |-- seg: string (nullable = true)
 |-- avg_all_costume_total_use: double (nullable = true)
 |-- group: string (nullable = false)



In [76]:
post_only_wsus_gcoin_by_user.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- avg_wsus_total_use: double (nullable = true)
 |-- avg_wsus_paid_use: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)
 |-- seg: string (nullable = true)
 |-- group: string (nullable = false)



In [207]:
post_df = post_only_costume_gcoin_by_user.withColumn("avg_wsus_total_use", lit(0)).withColumn("avg_wsus_paid_use", lit(0)) \
    .unionByName(post_costume_and_wsus_gcoin_by_user) \
    .unionByName(post_only_wsus_gcoin_by_user.withColumn("costume_type", lit("wsus")).withColumn("avg_total_use", lit(0)).withColumn("avg_paid_use", lit(0)).withColumn("avg_all_costume_total_use", lit(0)).withColumn("avg_chest_cnt", lit(0)))

In [208]:
post_df = post_df.withColumnRenamed("group", "post_group").withColumnRenamed("seg", "post_seg")

In [181]:
post_df.groupBy("post_group", "post_seg").agg(count("*")).show(truncate=False)

+----------+--------+--------+
|post_group|post_seg|count(1)|
+----------+--------+--------+
|c         |E       |5       |
|b         |B       |89706   |
|a         |B       |191938  |
|c         |B       |11783   |
|b         |A       |207652  |
|b         |E       |8458    |
|a         |C       |30724   |
|c         |D       |1011    |
|b         |D       |60620   |
|c         |A       |489264  |
|a         |D       |20212   |
|a         |A       |954415  |
|b         |C       |30083   |
|c         |C       |1838    |
|a         |E       |3701    |
+----------+--------+--------+



In [84]:
post_df.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- post_group: string (nullable = false)
 |-- costume_type: string (nullable = false)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- avg_chest_cnt: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)
 |-- avg_all_costume_total_use: double (nullable = true)
 |-- post_seg: string (nullable = true)
 |-- avg_wsus_total_use: double (nullable = true)
 |-- avg_wsus_paid_use: double (nullable = true)



In [209]:
mysql.drop_table("labs", "workshop_user_post_seg_tmp")
mysql.insert_table(post_df, "labs", "workshop_user_post_seg_tmp")

In [86]:
pre_df.printSchema()

root
 |-- pre_group: string (nullable = false)
 |-- account_id: string (nullable = true)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- pre_seg: string (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)



In [3]:
pre_df = mysql.read_table(spark, "labs", "workshop_user_pre_seg_tmp")
post_df = mysql.read_table(spark, "labs", "workshop_user_post_seg_tmp")

In [4]:
pre_df = pre_df.withColumnRenamed("avg_total_use", "pre_avg_total_use") \
    .withColumnRenamed("avg_paid_use", "pre_avg_paid_use") \
    .withColumnRenamed("avg_wsus_cnt", "pre_avg_wsus_cnt")

In [6]:
df = pre_df.join(post_df, "account_id", "full_outer")

In [14]:
df = df.withColumn("pre_group", when(col("pre_group") == "a", "Costume").when(col("pre_group") == "b", "WC").when(col("pre_group") == "c", "W").otherwise(lit(None))) \
    .withColumn("post_group", when(col("post_group") == "a", "Costume").when(col("post_group") == "b", "WC").when(col("post_group") == "c", "W").otherwise(lit(None)))

In [15]:
mysql.drop_table("labs", "workshop_user_segmentation")
mysql.insert_table(df, "labs", "workshop_user_segmentation")

In [8]:
df.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- pre_group: string (nullable = true)
 |-- pre_avg_total_use: double (nullable = true)
 |-- pre_avg_paid_use: double (nullable = true)
 |-- pre_seg: string (nullable = true)
 |-- pre_avg_wsus_cnt: double (nullable = true)
 |-- post_group: string (nullable = true)
 |-- costume_type: string (nullable = true)
 |-- avg_total_use: double (nullable = true)
 |-- avg_paid_use: double (nullable = true)
 |-- avg_chest_cnt: double (nullable = true)
 |-- avg_wsus_cnt: double (nullable = true)
 |-- avg_all_costume_total_use: double (nullable = true)
 |-- post_seg: string (nullable = true)
 |-- avg_wsus_total_use: double (nullable = true)
 |-- avg_wsus_paid_use: double (nullable = true)



In [9]:
df.where(col("costume_type") == "workshop").select(sum("avg_total_use") * 2).show(truncate=False)

+------------------------+
|(sum(avg_total_use) * 2)|
+------------------------+
|3.42876256E9            |
+------------------------+



In [13]:
df.where(col("costume_type") == "workshop").select(sum("avg_total_use") * 2).show(truncate=False)

+------------------------+
|(sum(avg_total_use) * 2)|
+------------------------+
|3.42876256E9            |
+------------------------+



In [186]:
post_gcoin.where(col("event_name").like("%workshop%")).select(sum("paid_use"), sum(col("paid_use") + col("free_use"))).show(truncate=False)

+-------------+--------------------------+
|sum(paid_use)|sum((paid_use + free_use))|
+-------------+--------------------------+
|2697671661   |3428762560                |
+-------------+--------------------------+



In [81]:
post_gcoin.where(col("event_name").like("%workshop%")).select(countDistinct("account_id").alias("pu")).show(truncate=False)

+------+
|pu    |
+------+
|997514|
+------+



In [187]:
post_gcoin.where(col("event_name").like("%wsus%")).select(sum("paid_use"), sum(col("paid_use") + col("free_use"))).show(truncate=False)

+-------------+--------------------------+
|sum(paid_use)|sum((paid_use + free_use))|
+-------------+--------------------------+
|2173488315   |2840657600                |
+-------------+--------------------------+



In [213]:
load_data_mart("pc", post_end_date, post_end_date, "user_master").where(col("lastlogindate") >= post_start_date).select(countDistinct("accountid").alias("au")).show(truncate=False)

+--------+
|au      |
+--------+
|14164852|
+--------+



### Costume유저들의 workshop chest *1 찍먹 유저

In [35]:
workshop_gcoin = post_gcoin.where(col("event_name").like("%workshop%"))

In [37]:
workshop_gcoin.select("product_id", "product_name").distinct().show(truncate=False)

+---------------------+------------------------------+
|product_id           |product_name                  |
+---------------------+------------------------------+
|itemdesc.14300002    |ARCHIVIST'S CHEST             |
|itemdesc.13000582    |ARCHIVIST'S CHEST BUNDLE (x10)|
|itemdesc.13000577    |HUNTER'S CHEST PACK (x1)      |
|itemdesc.13000581    |ARCHIVIST'S CHEST BUNDLE (x5) |
|itemdesc.13000580    |ARCHIVIST'S CHEST BUNDLE (x1) |
|itemdesc.13000578    |HUNTER'S CHEST PACK (x5)      |
|currencydesc.chestkey|KEY                           |
|itemdesc.14300001    |HUNTER'S CHEST                |
+---------------------+------------------------------+



In [69]:
costume_user = post_df.where((col("costume_type") == "workshop") & (col("post_group") == "a")).select("account_id").distinct()

In [70]:
one_item_buyer = workshop_gcoin.join(costume_user, "account_id").groupBy("account_id").agg(sum("qty").alias("qty")).where(col("qty") == 1)

In [71]:
workshop_gcoin.join(one_item_buyer, "account_id").groupBy("product_id", "product_name").agg(countDistinct("account_id").alias("pu")).orderBy("pu", ascending=False).toPandas()

,product_id,product_name,pu
0,itemdesc.13000580,ARCHIVIST'S CHEST BUNDLE (x1),89530
1,currencydesc.chestkey,KEY,82088
2,itemdesc.14300001,HUNTER'S CHEST,57543
3,itemdesc.13000577,HUNTER'S CHEST PACK (x1),32247
4,itemdesc.13000581,ARCHIVIST'S CHEST BUNDLE (x5),28980
5,itemdesc.14300002,ARCHIVIST'S CHEST,27510
6,itemdesc.13000582,ARCHIVIST'S CHEST BUNDLE (x10),5130
7,itemdesc.13000578,HUNTER'S CHEST PACK (x5),2719


### WC유저들의 workshop chest *1 찍먹 유저

In [72]:
wc_user = post_df.where((col("costume_type") == "workshop") & (col("post_group") == "b")).select("account_id").distinct()

In [73]:
wc_one_item_buyer = workshop_gcoin.join(wc_user, "account_id").groupBy("account_id").agg(sum("qty").alias("qty")).where(col("qty") == 1)

In [74]:
workshop_gcoin.join(wc_one_item_buyer, "account_id").groupBy("product_id", "product_name").agg(countDistinct("account_id").alias("pu")).orderBy("pu", ascending=False).toPandas()

,product_id,product_name,pu
0,currencydesc.chestkey,KEY,28008
1,itemdesc.13000580,ARCHIVIST'S CHEST BUNDLE (x1),24031
2,itemdesc.14300001,HUNTER'S CHEST,12614
3,itemdesc.13000581,ARCHIVIST'S CHEST BUNDLE (x5),6538
4,itemdesc.13000577,HUNTER'S CHEST PACK (x1),5466
5,itemdesc.14300002,ARCHIVIST'S CHEST,4784
6,itemdesc.13000582,ARCHIVIST'S CHEST BUNDLE (x10),3544
7,itemdesc.13000578,HUNTER'S CHEST PACK (x5),885


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 48990)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

## 이전 유저 세분화 day28 분석과 비교

In [19]:
before = mysql.read_table(spark, "labs", "workshop_user_seg_df_day28")
before = before.where(~col("post_seg").isNull()) \
    .withColumnRenamed("account_id", "before_account_id").withColumnRenamed("seg_group2", "before_post_group").withColumnRenamed("group2", "before_post_seg") \
    .select("before_account_id", "before_post_group", "before_post_seg").distinct()

In [20]:
after = mysql.read_table(spark, "labs", "workshop_user_segmentation")
after = after.where(~col("post_group").isNull()) \
    .select("account_id", "post_group", "post_seg").distinct()

In [21]:
compare_df = before.join(after, before.before_account_id == after.account_id, "full_outer")

In [23]:
mysql.insert_table(compare_df, "labs", "workshop_compare_df")

In [22]:
compare_df.groupBy("before_post_group", "before_post_seg", "post_group", "post_seg").agg(countDistinct("account_id").alias("user_cnt")).toPandas()

,before_post_group,before_post_seg,post_group,post_seg,user_cnt
0,None,None,Costume,B,39608
1,None,None,Costume,C,1921
2,a,B,WC,A,8423
3,None,None,Costume,D,817
4,b,A,WC,C,1106
5,a,A,W,C,71
6,a,B,W,B,109
7,a,D,WC,E,176
8,None,None,W,B,4840
9,c,A,WC,B,3192


SELECT pre_post_group, pre_post_seg, post_group, post_seg, count(distinct account_id)
FROM (
SELECT *
FROM (SELECT account_id as pre_account_id, seg_group2 as pre_post_group, group2 as pre_post_seg
	FROM labs.workshop_user_seg_df_day28
	WHERE post_seg is not null
	GROUP BY 1, 2, 3) a
FULL OUTER JOIN (SELECT account_id, post_group, post_seg
	FROM labs.workshop_user_segmentation
	WHERE post_group is not null
	GROUP BY 1, 2, 3) b ON a.pre_account_id = b.account_id
) tmp

### 잘팔린 다른상품과 2패치 기간 pur 비교

In [83]:
wsus01 = load_data_mart("pc", "2022-01-11", "2022-03-15", "gcoin_use").where(col("event_name") == "202201_wsus")
wsus01.select(countDistinct("account_id").alias('pu')).show(truncate=False)

+-------+
|pu     |
+-------+
|3006594|
+-------+



In [90]:
wsus01.select(sum('paid_use').alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+----------+----------+
|paid_use  |total_use |
+----------+----------+
|1072812415|4288861600|
+----------+----------+



In [84]:
wsus01_au = load_data_mart('pc', "2022-03-15", "2022-03-15", "user_master").where(col("lastlogindate") >= "2022-01-11")
wsus01_au.select(countDistinct("accountid").alias("au")).show(truncate=False)

+--------+
|ai      |
+--------+
|23803102|
+--------+



In [85]:
wsus02 = load_data_mart("pc", "2022-02-16", "2022-04-12", "gcoin_use").where(col("event_name") == "202202_wsus_progressive")
wsus02.select(countDistinct("account_id").alias('pu')).show(truncate=False)

+-------+
|pu     |
+-------+
|1115926|
+-------+



In [92]:
wsus02.select(sum('paid_use').alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+----------+----------+
|paid_use  |total_use |
+----------+----------+
|1326468091|2161443400|
+----------+----------+



In [40]:
from datetime import date, timedelta

start_date= date(2022, 2, 16)
end_date = date(2022, 4, 12)

l = pd.date_range(start_date, end_date,freq='d').tolist()
date_list = [datetime.strftime(i, "%Y-%m-%d") for i in l]

In [42]:
cum_pu_list = []
for date in date_list:
    cum_pu = wsus02.where(col("date") <= date).select(countDistinct("account_id")).collect()[0][0]
    cum_pu_list.append(cum_pu)

In [43]:
cum_pu_df = {
    "date": date_list,
    "cum_pu": cum_pu_list
}
cum_pu_pd = pd.DataFrame.from_dict(cum_pu_df)
cum_pu_pd.to_csv("./cum_wsus_pu_pd.csv")

In [86]:
wsus02_au = load_data_mart('pc', "2022-04-12", "2022-04-12", "user_master").where(col("lastlogindate") >= "2022-02-16")
wsus02_au.select(countDistinct("accountid").alias("au")).show(truncate=False)

+--------+
|ai      |
+--------+
|17112011|
+--------+



In [87]:
wsus06 = load_data_mart("pc", "2022-06-08", "2022-08-09", "gcoin_use").where(col("event_name") == "202206_wsus_progressive")
wsus06.select(countDistinct("account_id").alias('pu')).show(truncate=False)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39451)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39451)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

+------+
|pu    |
+------+
|710643|
+------+



In [93]:
wsus06.select(sum('paid_use').alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+----------+----------+
|paid_use  |total_use |
+----------+----------+
|1423976225|1942570200|
+----------+----------+



In [88]:
wsus06_au = load_data_mart('pc', "2022-08-09", "2022-08-09", "user_master").where(col("lastlogindate") >= "2022-06-08")
wsus06_au.select(countDistinct("accountid").alias("au")).show(truncate=False)

+--------+
|au      |
+--------+
|15390439|
+--------+



In [89]:
post_gcoin.where(col("event_name") == "202207_wsus").select(countDistinct("account_id").alias("pu")).show(truncate=False)

+------+
|pu    |
+------+
|345473|
+------+



In [94]:
post_gcoin.where(col("event_name") == "202207_wsus").select(sum('paid_use').alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+---------+---------+
|paid_use |total_use|
+---------+---------+
|425851880|623266400|
+---------+---------+



In [95]:
post_gcoin.where(col("event_name") == "202207_workshop").select(sum('paid_use').alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+----------+----------+
|paid_use  |total_use |
+----------+----------+
|2697671661|3428762560|
+----------+----------+



In [12]:
from datetime import date, timedelta

start_date= date(2022, 7, 13)
end_date = date(2022, 9, 6)

l = pd.date_range(start_date, end_date,freq='d').tolist()
date_list = [datetime.strftime(i, "%Y-%m-%d") for i in l]

In [19]:
cum_pu_list = []
workshop_gcoin = post_gcoin.where(col("event_name") == "202207_workshop")
for date in date_list:
    cum_pu = workshop_gcoin.where(col("date") <= date).select(countDistinct("account_id")).collect()[0][0]
    print(date, cum_pu)
    cum_pu_list.append(cum_pu)

2022-07-13 83321
2022-07-14 146317
2022-07-15 190777
2022-07-16 232117
2022-07-17 267676
2022-07-18 293632
2022-07-19 317870
2022-07-20 341720
2022-07-21 364153
2022-07-22 384922
2022-07-23 407068
2022-07-24 428207
2022-07-25 445269
2022-07-26 460930
2022-07-27 483618
2022-07-28 502247
2022-07-29 519027
2022-07-30 536474
2022-07-31 553533
2022-08-01 567983
2022-08-02 581062
2022-08-03 595099
2022-08-04 608290
2022-08-05 622158
2022-08-06 636991
2022-08-07 651894
2022-08-08 664454
2022-08-09 677314
2022-08-10 689554
2022-08-11 703276
2022-08-12 716773
2022-08-13 730786
2022-08-14 744638
2022-08-15 756614
2022-08-16 767665
2022-08-17 779397
2022-08-18 791030
2022-08-19 802360
2022-08-20 815205
2022-08-21 827965
2022-08-22 838342
2022-08-23 848414
2022-08-24 858499
2022-08-25 868172
2022-08-26 880867
2022-08-27 893554
2022-08-28 906108
2022-08-29 916870
2022-08-30 927246
2022-08-31 937277
2022-09-01 946859
2022-09-02 956584
2022-09-03 968207
2022-09-04 979129
2022-09-05 988551
2022-09-06 

In [20]:
cum_pu_df = {
    "date": date_list,
    "cum_pu": cum_pu_list
}
cum_pu_pd = pd.DataFrame.from_dict(cum_pu_df)
cum_pu_pd.to_csv("./cum_pu_pd.csv")

In [25]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

mySchema = StructType([ StructField("date", StringType(), True)\
                       ,StructField("cum_pu", IntegerType(), True)])
cum_pu_df = spark.createDataFrame(cum_pu_pd, schema = mySchema) 

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o84.sessionState

In [10]:
from pyspark.sql.window import Window
workshop_distinct_user_by_date = post_gcoin.where(col("event_name") == "202207_workshop").select("date", "account_id").distinct()

cum_count = size(
    collect_set(col('account_id'))
    .over(Window.orderBy("date")
             .rangeBetween(Window.unboundedPreceding, 0))
).alias('cum_count')
tmp_result = workshop_distinct_user_by_date.select("date", cum_count)